In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import GradScaler
from open_clip import ClipLoss

import sys
sys.path.append("/home/docker_current/py_files/finetune_exp/metaclip_arch")
sys.path.append('/home/docker_current/py_files/utils')
from metaclip_train import train_one_epoch, evaluate
from metaclip_train import cosine_lr

In [2]:
def load_embed(path, you_are_stupid=False, test_size=0.25):
    with open(path, 'rb') as f:
        text_feat_array = np.load(f)
        image_feat_array = np.load(f)

    X_train, X_test, y_train, y_test = train_test_split(text_feat_array, image_feat_array, 
                                                            test_size=test_size, random_state=42)

    if you_are_stupid:
        X_train, y_train  = np.vstack((X_train, y_train)), np.vstack((y_train, X_train))
        print("you are stupid")
    
    print("Shape X_train = ", X_train.shape)

    return X_train, X_test, y_train, y_test 

In [4]:
base_emb_path = "/home/docker_current/py_files/embeddings/"

name_emb_file_trans = base_emb_path + "trans_clip.npy"
X_train_trans, X_test_trans, y_train_trans, y_test_trans = load_embed(name_emb_file_trans)

name_emb_file_miltilang = base_emb_path + "Roberta_Large_Vit_L_14.npy"
X_train_ml, X_test_ml, y_train_ml, y_test_ml = load_embed(name_emb_file_miltilang)

X_train = np.hstack((X_train_trans, X_train_ml))
X_test = np.hstack((X_test_trans, X_test_ml))
y_train = np.hstack((y_train_trans, y_train_ml))
y_test = np.hstack((y_test_trans, y_test_ml))

print('Shape = ', X_train.shape)

Shape X_train =  (1573, 1024)
Shape X_train =  (1573, 768)
Shape =  (1573, 1792)


In [15]:
np.hstack((X_train, X_train,)).shape

(1573, 2048)